## Prepare evaluation

In [30]:
import json
import os
def get_image_files(folder_path):
    image_extensions = ['.jpeg','.jpg']

    image_files = []
    for file in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, file)):
            if any(file.lower().endswith(ext) for ext in image_extensions):
                image_files.append(file)

    return image_files
import pandas as pd
with open('gens.json','r')as f:
    gens=json.load(f)

import json

images=get_image_files('/home/coder/Abid/proj_with_marco/Data/train_data')
with open('all_caps.json','r') as f:
    caps=json.load(f)

labels={}
for k,v in zip(caps.keys(),caps.values()):
    labels[k]=v[-1]

In [31]:
data={}
for g in gens:
    
    img=g.split('_mid',1)[0].replace('.','_')
    generated_text=g.split('--  ',1)[1].split('.')[0]
    data[img]=[generated_text , labels[img],caps[img][0]]

data # a dictionary containing the llama generations , the real caption and the name of the class 

{'Walk_p04_r00_v05_c05': ['The person is standing in the living room, facing away from the camera, holding a remote control in',
  'The person is walking into the living room.',
  'Walk'],
 'Sitdown_p12_r01_v03_c04': ['The person is sitting on the couch',
  'The person is sitting on the couch.',
  'Sitdown'],
 'Walk_p10_r05_v13_c07': ['The person is standing at a counter, possibly preparing food or using a device',
  'The person is walking in the kitchen near a counter.',
  'Walk'],
 'Sitdown_p09_r03_v07_c04': ['The person is sitting on the green couch, facing the table with papers and a remote',
  'The person is sitting down on the couch.',
  'Sitdown'],
 'Sitdown_p10_r04_v03_c04': ['The person is sitting on the couch with their legs bent',
  'The person is attempting to sit down on the green couch.',
  'Sitdown'],
 'Drink_Frombottle_p06_r01_v10_c02': ['The person is seated at a table, appearing to engage in eating or drinking from a red cup',
  'the person is drinking from a bottle, 

## start evaluation

In [32]:
activity_dict = {
    'Cook_Cleandishes': ['cook', 'clean', 'dishes', 'cooking', 'washing','glass' ,'sink','preparing' ,'sandwich' ,'cleaning' ,'washing' ,'cup' , 'scrub', 'dry', 'kitchen', 'sink', 'utensils', 'wash', 'rinse', 'soapy', 'cleanup', 'appliances'],
    'Cook_Cleanup': ['cook','organiz','wip', 'dishe','dishwasher','loading','clean', 'up', 'cleaning', 'area' ,'counter' ,'cleaning' ,'table','up','cooking'  ,'stovetop','stove','washing' ,'cooking', 'tidy', 'kitchen', 'organize', 'scrub', 'wipe', 'disinfect', 'stove', 'counter', 'spotless', 'neat'],
    'Cook_Cut': ['cook', 'chopping' ,'vegetable' ,'counter','cutting' ,'board' ,'cutting' ,'cooking' ,'preparing' ,'food','slicing', 'slic','cut', 'cooking', 'chop', 'cutting', 'slice', 'dice', 'knife', 'board', 'vegetables', 'mince', 'food', 'prepare', 'ingredients', 'sharp'],
    'Cook_Stir': ['cook','stirring' ,'pot' ,'ladle' ,'cooking' ,'ingredients' ,'stove','bowl','pouring','container' , 'stir', 'cooking', 'mix', 'stirring', 'whisk', 'spoon', 'bowl', 'combine', 'pot', 'blend', 'heat', 'sauce', 'stove', 'ingredients'],
    'Cook_Usestove': ['cook', 'cooking' ,'stove' ,'oven' ,'preparing' ,'food' ,'use', 'stove', 'cooking', 'heat','cook', 'use', 'stove', 'cooking', 'heat', 'burner', 'pot', 'pan', 'kitchen', 'boil', 'simmer', 'bake', 'fry', 'flame', 'prepare'],
    'Cutbread': ['cut', 'bread', 'slice', 'cutting', 'knife', 'loaf', 'toast', 'slicing', 'butter', 'sharp', 'crust', 'crumbs', 'kitchen', 'board', 'baking'],
    'Drink_Frombottle': ['drink', 'bottle', 'drinking', 'water', 'sip', 'cap', 'beverage', 'gulp', 'refresh', 'cold', 'thirsty', 'plastic', 'liquid', 'carry', 'portable'],
    'Drink_Fromcan': ['drink', 'can', 'drinking', 'soda', 'pop', 'beverage', 'aluminum', 'carbonated', 'sip', 'cold', 'thirst', 'fizzy', 'gulp', 'refresh', 'cola'],
    'Drink_Fromcup': ['drink', 'cup', 'drinking', 'water', 'sip', 'mug', 'beverage', 'ceramic', 'hot', 'cold', 'tea', 'coffee', 'handle', 'pour', 'refresh'],
    'Drink_Fromglass': ['drink', 'glass', 'drinking', 'water', 'juice', 'sip', 'beverage', 'clear', 'cold', 'thirsty', 'liquid', 'refresh', 'ice', 'wine', 'serve'],
    'Eat_Attable': ['eat', 'table', 'eating', 'meal', 'dining', 'plate', 'food', 'fork', 'spoon', 'chairs', 'family', 'lunch', 'dinner', 'breakfast', 'restaurant'],
    'Eat_Snack': ['eat', 'snack', 'eating', 'bite', 'munch', 'chips', 'cookie', 'crackers', 'small', 'quick', 'candy', 'popcorn', 'treat', 'pack', 'nibble'],
    'Enter': ['enter', 'walk','standing' ,'moving' ,'move','entering', 'come', 'arrival', 'step', 'door', 'walk', 'threshold', 'room', 'hallway', 'pass', 'open', 'welcome', 'start', 'arrive'],
    'Getup': ['get', 'up', 'stand', 'rise', 'standing', 'awake', 'morning', 'stretch', 'bed', 'start', 'day', 'move', 'wake', 'active', 'energize'],
    'Laydown': ['lay', 'down','relaxing ', 'sitting' ,'rest', 'sleep', 'lying', 'relax', 'bed', 'sofa', 'tired', 'nap', 'recline', 'comfort', 'flat', 'repose', 'quiet'],
    'Leave': ['leave','leaving', 'exit', 'go','standing ','entering', 'depart', 'walk', 'away', 'move', 'farewell', 'bye', 'door', 'out', 'goodbye', 'finish', 'vacate', 'end'],
    'Makecoffee_Pourgrains': ['pour','making','pouring','make', 'coffee', 'pour', 'grains', 'beans', 'brew', 'aroma', 'hot', 'drink', 'caffeine', 'cup', 'grind', 'filter', 'pot', 'kitchen'],
    'Makecoffee_Pourwater': ['make','filling' ,'drink ','coffee', 'pour', 'water', 'brew', 'hot', 'steam', 'pot', 'boil', 'cup', 'filter', 'kettle', 'aroma', 'grind', 'beverage'],
    'Maketea_Boilwater': ['make', 'tea', 'boil', 'water', 'brew', 'kettle', 'cup', 'hot', 'steam', 'bag', 'loose', 'leaves', 'pour', 'aroma', 'drink'],
    'Maketea_Insertteabag': ['make', 'tea', 'insert', 'teabag', 'cup', 'brew', 'hot', 'water', 'steam', 'string', 'bag', 'kettle', 'flavor', 'dip', 'aroma'],
    'Pour_Frombottle': ['pour', 'bottle', 'pouring', 'liquid', 'glass', 'water', 'wine', 'juice', 'beverage', 'spill', 'flow', 'drink', 'container', 'cap', 'refresh'],
    'Pour_Fromcan': ['pour', 'can', 'pouring', 'soda', 'beverage', 'carbonated', 'drink', 'spill', 'aluminum', 'flow', 'pop', 'refresh', 'open', 'fizzy', 'gulp'],
    'Pour_Fromkettle': ['pour','water','cup', 'kettle', 'tea', 'pouring', 'boil', 'water', 'hot', 'brew', 'drink', 'steam', 'cup', 'aroma', 'handle', 'spout', 'heat'],
    'Readbook': ['read', 'book','paper', 'liquid', 'reading', 'story', 'novel', 'pages', 'literature', 'library', 'study', 'fiction', 'learn', 'chapter', 'words', 'relax', 'imagine'],
    'Sitdown': ['sit', 'down', 'sitting', 'rest', 'chair', 'sofa', 'seat', 'relax', 'position', 'cushion', 'pause', 'stool', 'comfort', 'recline', 'quiet'],
    'Takepills': ['take','drink','medication','pill','sip', 'pills', 'medicine', 'tablet', 'swallow', 'capsule', 'dose', 'prescription', 'health', 'water', 'pharmacy', 'bottle', 'drug', 'relief', 'treatment'],
    'Uselaptop': ['use','monitor','sit','table', 'laptop', 'computer', 'typing', 'keyboard', 'screen', 'device', 'work', 'technology', 'study', 'portable', 'task', 'mouse', 'coding', 'browse'],
    'Usetablet': ['use', 'tablet', 'screen', 'device', 'touch', 'portable', 'apps', 'draw', 'browse', 'technology', 'stylus', 'digital', 'read', 'lightweight', 'entertainment'],
    'Usetelephone': ['use','phone','device', 'telephone', 'call', 'talk', 'dial', 'ring', 'speak', 'conversation', 'voice', 'device', 'number', 'mobile', 'communication', 'contact', 'chat'],
    'Walk': ['walk','standing','stand', 'enter','walking', 'step', 'move', 'stride', 'path', 'pace', 'travel', 'exercise', 'legs', 'outdoor', 'distance', 'stroll', 'hike', 'active'],
    'WatchTV': ['watch','living room', 'TV', 'television', 'viewing', 'screen', 'remote', 'show', 'program', 'movie', 'series', 'channel', 'entertainment', 'couch', 'relax', 'media']
}

In [33]:
# import json
# with open('cs_split.json','r') as f:
#     to_be_used=json.load(f)


# not_used=[]
# used=[]
# len(to_be_used)

cap=[data[img][1] for img in list(data.keys())]
llama=[data[img][0] for img in  list(data.keys())]
gt=[data[img][2] for img in  list(data.keys())]

In [34]:
import pandas as pd
eval=pd.DataFrame({'image':list(data.keys()),'ground_truth':gt,'llama':llama,'captions':cap})
eval

,image,ground_truth,llama,captions
0,Walk_p04_r00_v05_c05,Walk,"The person is standing in the living room, fac...",The person is walking into the living room.
1,Sitdown_p12_r01_v03_c04,Sitdown,The person is sitting on the couch,The person is sitting on the couch.
2,Walk_p10_r05_v13_c07,Walk,"The person is standing at a counter, possibly ...",The person is walking in the kitchen near a co...
3,Sitdown_p09_r03_v07_c04,Sitdown,"The person is sitting on the green couch, faci...",The person is sitting down on the couch.
4,Sitdown_p10_r04_v03_c04,Sitdown,The person is sitting on the couch with their ...,The person is attempting to sit down on the gr...
...,...,...,...,...
125,Sitdown_p14_r05_v02_c04,Sitdown,"The person is sitting on the couch, holding a ...",The person is sitting down on the couch and pl...
126,Laydown_p10_r02_v03_c04,Laydown,"The person is lying on the couch, covered with...","The person is lying on the couch, covered with..."
127,Drink_Frombottle_p09_r00_v07_c04,Drink_Frombottle,The person is drinking from a cup while seated...,The person is drinking from a bottle while sit...
128,Drink_Frombottle_p04_r00_v07_c04,Drink_Frombottle,"The person is sitting on the couch, holding an...",The person is holding a small object near thei...


In [35]:

llama_counter,capcounter=0,0

llama_mpca={}


for i in range(len(eval)):
    if eval['image'][i] in to_be_used:
        if eval['ground_truth'][i] in activity_dict:
            dpsk_y,intern2kf_y,llama_y,cap_y=0,0,0,0
            for keyword in activity_dict[eval['ground_truth'][i]]:
                

                if not llama_y:
                    if keyword in eval['llama'][i]:
                        llama_counter+=1
                        llama_y=1
                        try: 
                            llama_mpca[eval['ground_truth'][i]]+=1
                        except:
                            llama_mpca[eval['ground_truth'][i]]=1
                        #print('llama --',eval['ground_truth'][i],'---',keyword,'---',eval['llama'][i])

                if not cap_y:
                    if keyword in eval['captions'][i]:
                        capcounter+=1
                        used.append(sorted(caps)[i])
                        cap_y=1
                

            used.append(eval['image'][i])
            

    else:
        not_used.append(eval['image'][i])
                
                
print('\n \n \n \n Results :  Correct model generations  \n     -  llama: ',llama_counter, '-  captions',capcounter, '\n \n \n') 


 
 
 
 Results :  Correct model generations  
     -  llama:  71 -  captions 78 
 
 



In [36]:
correct_counts = dpsk_mpca
ground_truth_counts = dict(eval['ground_truth'].value_counts())

# Compute per-class accuracy and then mean
accuracies = []
for cls, total in ground_truth_counts.items():
    correct = correct_counts.get(cls, 0)
    acc = correct / total
    accuracies.append(acc)

mpca = sum(accuracies) / len(accuracies)
print(f"Mean Per Class Accuracy (MPCA): {mpca:.4f} or {mpca * 100:.2f}%")


Mean Per Class Accuracy (MPCA): 0.0000 or 0.00%
